# Churning prediction

This model training is aim to predict pet owners who will likely to leave the platform in the future.

### Setting up dataset for training

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine

DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "dbt_db",
    "user": "dbt_user",
    "password": "dbt_pass"
}

# Get the dataset from the database
engine = create_engine(f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
df = pd.read_sql_query("SELECT * FROM petcare_features.features_churn_dataset", engine)

# Prepare the features for training
# Remove features that are used to calculate the target flag
feature_cols = [col for col in df.columns if col not in ['owner_id', 'churn_flag', 'total_consultations', 'days_since_last_consultation']]
x_ax = df[feature_cols].fillna(0)
y_ax = df['churn_flag']

# Split the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(x_ax, y_ax, test_size=0.2, random_state=42, stratify=y_ax)

In [37]:
import mlflow

MLFLOW_TRACKING_URI = "http://localhost:5002"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

mlflow.set_experiment("churning_prediction")

2026/01/03 00:51:22 INFO mlflow.tracking.fluent: Experiment with name 'churning_prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///tmp/mlflow/artifacts/1', creation_time=1767376282847, experiment_id='1', last_update_time=1767376282847, lifecycle_stage='active', name='churning_prediction', tags={}>

### Experiment #1: Logistic Regression

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)
import numpy as np
import mlflow
import pickle
from mlflow.models.signature import infer_signature

mlflow.autolog(log_models=False)

with mlflow.start_run(run_name="logistic_regression_model"):
    params = {
        "random_state": 42,
        "max_iter": 1000
    }
    
    # Initialize the model
    lr_model = LogisticRegression(**params)

    # Train the model
    lr_model.fit(X_train, y_train)

    # Make predictions
    y_pred = lr_model.predict(X_test)
    y_pred_proba = lr_model.predict_proba(X_test)[:, 1]

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred_proba) if len(np.unique(y_test)) > 1 else 0

    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC AUC Score: {roc_auc}")
    
    signature = infer_signature(X_train, lr_model.predict(X_train))
    
    mlflow.log_params(params)
    mlflow.log_metrics({
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc
    })
    mlflow.sklearn.log_model(lr_model, 
        registered_model_name="ChurnPredictionModel",
        name="logistic_regression_model",
        input_example=X_train,
        signature=signature,
        serialization_format=SERIALIZATION_FORMAT_CLOUDPICKLE
        )

2026/01/03 00:54:20 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2026/01/03 00:54:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/pkhantiviriya/petcare-data-pipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2026/01/03 00:54:20 WARNING 

ROC AUC Score: 0.7000000000000002
Accuracy: 0.725
Precision: 0.6428571428571429
Recall: 0.6
F1 Score: 0.6206896551724138
ROC AUC Score: 0.848


2026/01/03 00:54:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/pkhantiviriya/petcare-data-pipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
Successfully registered model 'ChurnPredictionModel'.
2026/01/03 00:54:22 INFO mlflow.store.model_registry.abstract_store: 

🏃 View run logistic_regression_model at: http://localhost:5002/#/experiments/1/runs/79331c17e48a45459fcec50fbfcb8cfa
🧪 View experiment at: http://localhost:5002/#/experiments/1


Created version '1' of model 'ChurnPredictionModel'.


### Experiment #2 XGBoost

In [39]:
from mlflow.sklearn import SERIALIZATION_FORMAT_CLOUDPICKLE
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score
)
import numpy as np
import mlflow
import pickle
from mlflow.models.signature import infer_signature

mlflow.xgboost.autolog(log_models=False)

with mlflow.start_run(run_name="xgboost_model") as run:
    params = {
        "random_state": 42,
        "n_estimators": 100,
        "max_depth": 6
    }
    
    # Initialize the model
    xgb_model = XGBClassifier(**params)

    # Train the model
    xgb_model.fit(X_train, y_train)

    # Make predictions
    y_pred = xgb_model.predict(X_test)
    y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred_proba) if len(np.unique(y_test)) > 1 else 0

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC AUC Score: {roc_auc}")

    signature = infer_signature(X_train, xgb_model.predict(X_train))

    mlflow.log_params(params)
    mlflow.log_metrics({
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc
    })
    mlflow.sklearn.log_model(xgb_model,
        registered_model_name="ChurnPredictionModel",
        name="xgboost_model",
        input_example=X_train,
        )


2026/01/03 00:54:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/pkhantiviriya/petcare-data-pipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2026/01/03 00:54:35 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/pkhantiviriya

Accuracy: 0.775
Precision: 0.6666666666666666
Recall: 0.8
F1 Score: 0.7272727272727273
ROC AUC Score: 0.8693333333333333


2026/01/03 00:54:37 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/pkhantiviriya/petcare-data-pipeline/.venv/lib/python3.12/site-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
Registered model 'ChurnPredictionModel' already exists. Creating a new version of this model...
2026/01/03 00:54:37 INFO ml

🏃 View run xgboost_model at: http://localhost:5002/#/experiments/1/runs/480c999438ef48c6ba06a3510e9c3e6e
🧪 View experiment at: http://localhost:5002/#/experiments/1


Created version '2' of model 'ChurnPredictionModel'.
